# UNSW-NB15 Full Dataset Preparation (Local Execution)
This notebook prepares the full UNSW-NB15 dataset from the 4 split parts and GT labels. It downsamples benign traffic, filters columns, and prepares the data for model training.

## Load and Combine Data Parts, add on the missing headers

In [ ]:

import pandas as pd
from pathlib import Path
COLUMNS = [
    'id','dur','proto','service','state','spkts','dpkts','sbytes','dbytes','rate',
    'sttl','dttl','sload','dload','sloss','dloss','sinpkt','dinpkt','sjit','djit',
    'swin','stcpb','dtcpb','dwin','tcprtt','synack','ackdat','smean','dmean',
    'trans_depth','response_body_len','ct_srv_src','ct_state_ttl','ct_dst_ltm',
    'ct_src_dport_ltm','ct_dst_sport_ltm','ct_dst_src_ltm','is_ftp_login',
    'ct_ftp_cmd','ct_flw_http_mthd','ct_src_ltm','ct_srv_dst','is_sm_ips_ports',
    'attack_cat','label'
]               # ← Column names from the dataset documentation, used as headers

# CONFIG: Set the path to your dataset folder
DATA_DIR = Path("/Users/jasmine/Documents/CSCI7783_Information Security/Project1_Data/NUSW-NB15/data")  # ← Change this!
csv_parts = [f"NUSW-NB15_{i}.csv" for i in range(1, 5)]

# Load and combine CSV files with column names as headers
df_list = [pd.read_csv(DATA_DIR / part, names=COLUMNS, header=0) for part in csv_parts]

first_row = df_list[0].iloc[[0]]  # Keep the first row from the first part
df_combined = pd.concat(df_list, ignore_index=True)
df_combined = pd.concat([first_row, df_combined], ignore_index=True)
print("length of columns:", len(df_combined.columns))

# Nicely formatted preview
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 0)  # auto-fit
pd.set_option("display.width", 0)
pd.set_option("display.max_colwidth", 200)

print("\n[check] first 22 rows:")
display(df_combined.head(22))
len(df_rows := df_combined)
print(f"\n[info] total rows: {len(df_rows)}")



## Keep Only Required Rows

In [ ]:
mask_nan = df_combined.isna().any(axis=1)
to_drop = df_combined[mask_nan].sample(frac=0.6, random_state=42).index
df_combined = df_combined.drop(index=to_drop).reset_index(drop=True)

df_combined = df_combined.fillna("Normal")

print("Remaining rows:", len(df_combined))

## Save Cleaned Data to CSV. ZIP

In [ ]:

OUTPUT_CSV = DATA_DIR / "UNSW_NB15_cleaned.csv"
OUTPUT_ZIP = DATA_DIR / "UNSW_NB15_cleaned.csv.zip"

df_combined.to_csv(OUTPUT_CSV, index=False)
df_combined.to_csv(OUTPUT_ZIP, index=False, compression={
    "method": "zip",
    "archive_name": OUTPUT_CSV.name
})

print("[save] CSV ->", OUTPUT_CSV, "size:", OUTPUT_CSV.stat().st_size, "bytes")
print("[save] ZIP ->", OUTPUT_ZIP, "size:", OUTPUT_ZIP.stat().st_size, "bytes")

## Define x and y, then Split into Train/Test

In [ ]:

from sklearn.model_selection import train_test_split

import pandas as pd

df = df_combined.copy()
# Drop non-predictive columns
df = df.drop(columns=['id', 'label'])
# Separate features and target
X = df.drop(columns=['attack_cat'])
y = df['attack_cat']
# One-hot encode categorical features
X = pd.get_dummies(X, drop_first=True)

# Train/test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

print("X_train:", X_train.shape, "| X_test:", X_test.shape)
print("y_train distribution:", y_train.value_counts(normalize=True).round(3).to_dict())
